In [9]:
!pip3 install --no-cache-dir tensorflow-rocm

     |████████████████████████████████| 281.0 MB 25.0 MB/s 
     |████████████████████████████████| 26.0 MB 13.6 MB/s 


In [2]:
!pip3 install tensorflow

     |████████████████████████████████| 320.5 MB 1.2 kB/s 


In [3]:
!pip3 install numpy
!pip3 install nltk

In [5]:
import math
import numpy as np
import string
import json
import os
import sys
import logging
import nltk
import random
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize

import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow.keras import backend as K

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

In [ ]:
print(device_lib.list_local_devices())

In [6]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/vicotrbb/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vicotrbb/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [31]:
chars = ''
maxlen = 60
step = 3

In [7]:
print(tf.keras.__version__)
print(tf.__version__)

2.4.0
2.3.1


In [8]:
# Fix seed's
os.environ['PYTHONHASHSEED']=str(66)
tf.random.set_seed(66)
np.random.seed(66)
random.seed(66)

In [28]:
def prepareData(dataFile, parts):
    f = open(dataFile,)
    data = json.load(f)

    content = list(data[x] for x in data.keys())
    text = ''

    for c in content[:parts]:
        for i in c:
            text += i

    print(f'Corpus length: {len(text)}')

    tokens = word_tokenize(text)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]

    text = ''
    for c in words:
        text += c
        text += ' '
    text = text.strip()

    print(f'Finished to load file')
    return text

In [29]:
text = prepareData('Carreira/PROJETOS/PESSOAIS/machine_learning/projects/wikipedia-nlp/wikipedia-content-dataset.json', 250)

Corpus length: 901824
Finished to load file


In [32]:
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Numero de sequencias:', len(sentences))

chars = sorted(list(set(text)))
print('Caracteres unicos:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

print('Vetorizando o texto')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
print('Finalizado vetorização do texto')

Numero de sequencias: 207132
Caracteres unicos: 254
Vetorizando o texto


In [33]:
print(x.shape)
print(y.shape)

(207132, 60, 254)
(207132, 254)


In [37]:
model = Sequential([
    LSTM(len(chars), return_sequences=True, input_shape=(maxlen, len(chars))),
    LSTM(len(chars), return_sequences=True),
    LSTM(len(chars)),
    Dense(len(chars), activation='relu'),
    Dense(len(chars), activation='softmax')
])
model.summary()

Model: &quot;sequential_1&quot;
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 60, 254)           517144    
_________________________________________________________________
lstm_4 (LSTM)                (None, 60, 254)           517144    
_________________________________________________________________
lstm_5 (LSTM)                (None, 254)               517144    
_________________________________________________________________
dense_2 (Dense)              (None, 254)               64770     
_________________________________________________________________
dense_3 (Dense)              (None, 254)               64770     
Total params: 1,680,972
Trainable params: 1,680,972
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [41]:
print("Treino do modelo")
model.fit(x, y, batch_size=128, epochs=75)

Treino do modelo
Epoch 1/75
1619/1619 [==============================] - 892s 551ms/step - loss: 2.6873 - accuracy: 0.1975
Epoch 2/75
1619/1619 [==============================] - 876s 541ms/step - loss: 2.3570 - accuracy: 0.2764
Epoch 3/75
1619/1619 [==============================] - 878s 543ms/step - loss: 2.1728 - accuracy: 0.3349
Epoch 4/75
1619/1619 [==============================] - 884s 546ms/step - loss: 2.0028 - accuracy: 0.3889
Epoch 5/75
1619/1619 [==============================] - 876s 541ms/step - loss: 1.8670 - accuracy: 0.4318
Epoch 6/75
1616/1619 [============================&gt;.] - ETA: 1s - loss: 1.7648 - accuracy: 0.4629

KeyboardInterrupt: 